In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
ham_dim = 10 # dimension of hamiltonian
renorm_ham_dim = 3 # dimension of renormalized hamiltonian

# hamiltonian
H = tf.complex(tf.random.normal((ham_dim, ham_dim), dtype=tf.float64),
               tf.random.normal((ham_dim, ham_dim), dtype=tf.float64))
H = 0.5 * (H + tf.linalg.adjoint(H))

# isometric matrix
q_in = tf.complex(tf.random.normal((ham_dim, renorm_ham_dim), dtype=tf.float64),
               tf.random.normal((ham_dim, renorm_ham_dim), dtype=tf.float64))
q_in, _ = tf.linalg.qr(q_in)
q_in = complex_to_real(q_in)

# min of loss function
min_value = tf.cast(tf.reduce_sum(tf.linalg.eigvalsh(H)[:renorm_ham_dim]),
                   dtype=tf.float64)

# riemannian optimizaers
opts = [StiefelSGD(0.03), StiefelSGD(0.03, 0.9)]

In [0]:
list_of_losses = []
# loop over optimizers
for opt in opts:
    # training loop
    losses = []
    q = tf.Variable(q_in)
    for _ in range(500):
        with tf.GradientTape() as tape:
            Z = tf.complex(q[..., 0], q[..., 1])
            loss = tf.cast(tf.linalg.trace((tf.linalg.adjoint(Z) @ H @ Z)),
                        dtype=tf.float64)
            losses.append(loss - min_value)
        grad = tape.gradient(loss, q)
        opt.apply_gradients(zip([grad], [q]))
    list_of_losses.append(losses)

In [0]:
# learning curves
plt.plot(list_of_losses[0], 'r')
plt.plot(list_of_losses[1], 'b')
plt.yscale('log')
plt.xlabel('iter')
plt.ylabel('err')